## Pregnancy Predictor

The purpose of this program is to create a application that would allow a person to fill in any amount of information they have on their fertility and recieve a prediction of their likelihood of getting pregnant. 

- I am using Bayes' theorem which finds the probability based on prior knowledge of any conditions that are related to the question. 

- For the pregnancy predictions I am using the three main methods of achieving pregnancy; natural pregnancy where sperm is introduced in the vagina, intrauterine insemination (IUI) where sperm is introduced directly into the uterus, and in vitro fertilization (IVF) where an already fertilized egg is implanted into the uterus. 

- Using national data for artifical reproductive technology (ART) I create the general chance of pregnancy for each method and then with each personal data points a person has their prediction will change for better accuracy. 

#### Importing libraries and Frameworks

I will be using NumPy and Flask

In [ ]:
import numpy as np
from flask import Flask, request, render_template

#### Using data from the Society for Assisted Reproductive Technology (SART) and open source data from the CDC on ART, we know the following
- Success rate for natural pregnancy during ovulation for an individual is 15-20%, with 80% of people becoming pregnant within the first 12 months of trying. 
- Success rate for medicated IUI is 20-20%, with 90% of people becoming pregnant within the first 4 tries. 
- Success rate for IVF is 30%, with each attempt increasing the likelihood. 

**_Note: These numbers are very generalized and do not take into account many different factors that affect pregnancy but they are the starting point._**

With these numbers I will set the learning rate for each method and create a class for Pregnancy Method. 

In [ ]:
class PregnancyMethod:
    def __init__(self, lr, max_attempts, success_rate):
        self.lr = lr
        self.max_attempts = max_attempts
        self.success_rate = success_rate
    
    def get_success_prob(self, num_attempts):
        if num_attempts > self.max_attempts:
            return self.success_rate
        else:
            return 1 - ((1 - self.lr) ** num_attempts)

natural = PregnancyMethod(lr=0.15, max_attempts=12, success_rate=0.8)
iui = PregnancyMethod(lr=0.225, max_attempts=4, success_rate=0.9)
ivf = PregnancyMethod(lr=0.3, max_attempts=float('inf'), success_rate=float('inf'))

#### Next I will create a class for the patient information. 

Important patient information for predicting pregnancy is
- age
- age of partner if the patient has a partner who produces sperm
- Anti-mullerian Hormone (AMH) which can interpret how many eggs a person has in reserve
- Follicular-stimulating Hormone (FSH) which can predict menopause
- cycle length which is important to know whether a person's cycle is consistent
- luteal phase which also shows consistancy in pregnancy and is important for natural conception timing
- past attempts which will relate to pregnancy method success
- past pregnancies
- past miscarriages

In [ ]:
class Patient:
    def __init__(self, age, partner_age, past_pregnancies, past_miscarriages, past_attempts, amh_level, fsh_level, cycle_length, luteal_phase):
        self.age = age
        self.partner_age = partner_age
        self.past_pregnancies = past_pregnancies
        self.past_miscarriages = past_miscarriages
        self.past_attempts = past_attempts
        self.amh_level = amh_level
        self.fsh_level = fsh_level
        self.cycle_length = cycle_length
        self.luteal_phase = luteal_phase

    def _get_success_prob(self, method):
        return method.get_success_prob(self.past_attempts)

    def natural_conception(self):
        return self._get_success_prob(natural)

    def iui_conception(self):
        return self._get_success_prob(iui)

    def ivf_conception(self):
        return self._get_success_prob(ivf)

    def natural_time(self):
        success_prob = self.natural_conception()
        time = -1 / np.log(1 - success_prob)
        return time

    def iui_time(self):
        success_prob = self.iui_conception()
        time = -1 / np.log(1 - success_prob)
        return time

    def ivf_time(self):
        success_prob = self.ivf_conception()
        time = -1 / np.log(1 - success_prob)
        return time

#### Next we will work on the prediction using Bayes Theorem. 

In [ ]:
def predict(patient, prior_probs):
    p = patient(patient['age'], patient['partner_age'], patient['past_pregnancies'], patient['past_miscarriages'], patient['past_attempts'], patient['amh_level'], patient['fsh_level'], patient['cycle_length'], patient['luteal_phase'])
    
    # Get likelihoods for each method
    natural_likelihood = p.natural_conception()
    iui_likelihood = p.iui_conception()
    ivf_likelihood = p.ivf_conception()
    
    # Update priors with likelihoods using Bayes' theorem
    natural_posterior = prior_probs['natural'] * natural_likelihood
    iui_posterior = prior_probs['iui'] * iui_likelihood
    ivf_posterior = prior_probs['ivf'] * ivf_likelihood
    
    # Normalize posteriors to obtain probabilities
    total = natural_posterior + iui_posterior + ivf_posterior
    natural_prob = natural_posterior / total
    iui_prob = iui_posterior / total
    ivf_prob = ivf_posterior / total
    
    # Get expected times for each method
    natural_time = p.natural_time()
    iui_time = p.iui_time()
    ivf_time = p.ivf_time()
    
    return {
        'natural_prob': natural_prob,
        'iui_prob': iui_prob,
        'ivf_prob': ivf_prob,
        'natural_time': natural_time,
        'iui_time': iui_time,
        'ivf_time': ivf_time
    }

#### Using Flask to set up an application 

In [ ]:
app = Flask(__name__)

class PregnancyMethod:
    def __init__(self, lr, max_attempts, success_rate):
        self.lr = lr
        self.max_attempts = max_attempts
        self.success_rate = success_rate
    
    def get_success_prob(self, num_attempts):
        if num_attempts > self.max_attempts:
            return self.success_rate
        else:
            return 1 - ((1 - self.lr) ** num_attempts)

natural = PregnancyMethod(lr=0.15, max_attempts=12, success_rate=0.8)
iui = PregnancyMethod(lr=0.225, max_attempts=4, success_rate=0.9)
ivf = PregnancyMethod(lr=0.3, max_attempts=float('inf'), success_rate=float('inf'))

class Patient:
    def __init__(self, age, partner_age, past_pregnancies, past_miscarriages, past_attempts, amh_level, fsh_level, cycle_length, luteal_phase):
        self.age = age
        self.partner_age = partner_age
        self.past_pregnancies = past_pregnancies
        self.past_miscarriages = past_miscarriages
        self.past_attempts = past_attempts
        self.amh_level = amh_level
        self.fsh_level = fsh_level
        self.cycle_length = cycle_length
        self.luteal_phase = luteal_phase

    def _get_success_prob(self, method):
        return method.get_success_prob(self.past_attempts)

    def natural_conception(self):
        return self._get_success_prob(natural)

    def iui_conception(self):
        return self._get_success_prob(iui)

    def ivf_conception(self):
        return self._get_success_prob(ivf)

    def natural_time(self):
        success_prob = self.natural_conception()
        time = -1 / np.log(1 - success_prob)
        return time

    def iui_time(self):
        success_prob = self.iui_conception()
        time = -1 / np.log(1 - success_prob)
        return time

    def ivf_time(self):
        success_prob = self.ivf_conception()
        time = -1 / np.log(1 - success_prob)
        return time

def predict(patient, prior_probs):
    p = Patient(patient['age'], patient['partner_age'], patient['past_pregnancies'], patient['past_miscarriages'], patient['past_attempts'], patient['amh_level'], patient['fsh_level'], patient['cycle_length'], patient['luteal_phase'])
    
    # Get likelihoods for each method
    natural_likelihood = p.natural_conception()
    iui_likelihood = p.iui_conception()
    ivf_likelihood = p.ivf_conception()
    
    # Update priors with likelihoods using Bayes' theorem
    natural_posterior = prior_probs['natural'] * natural_likelihood
    iui_posterior = prior_probs['iui'] * iui_likelihood
    ivf_posterior = prior_probs['ivf'] * ivf_likelihood
    
    # Normalize posteriors to obtain probabilities
    total = natural_posterior + iui_posterior + ivf_posterior
    natural_prob = natural_posterior / total
    iui_prob = iui_posterior / total
    ivf_prob = ivf_posterior / total
    
    # Get expected times for each method
    natural_time = p.natural_time()
    iui_time = p.iui_time()
    ivf_time = p.ivf_time()


#### Creating the HTML form

In [ ]:
<!DOCTYPE html>
<html>
<head>
	<title>Pregnancy Prediction</title>
</head>
<body>
	<form action="/" method="post">
		<label for="age">Age:</label>
		<input type="number" id="age" name="age" required><br>

		<label for="partner_age">Partner's Age:</label>
		<input type="number" id="partner_age" name="partner_age" required><br>

		<label for="past_pregnancies">Number of Past Pregnancies:</label>
		<input type="number" id="past_pregnancies" name="past_pregnancies" required><br>

		<label for="past_miscarriages">Number of Past Miscarriages:</label>
		<input type="number" id="past_miscarriages" name="past_miscarriages" required><br>

		<label for="past_attempts">Number of Past Conception Attempts:</label>
		<input type="number" id="past_attempts" name="past_attempts" required><br>

		<label for="amh_level">AMH Level:</label>
		<input type="number" step="0.01" id="amh_level" name="amh_level" required><br>

		<label for="fsh_level">FSH Level:</label>
		<input type="number" step="0.01" id="fsh_level" name="fsh_level" required><br>

		<label for="cycle_length">Menstrual Cycle Length:</label>
		<input type="number" id="cycle_length" name="cycle_length" required><br>

		<label for="luteal_phase">Luteal Phase Length:</label>
		<input type="number" id="luteal_phase" name="luteal_phase" required><br>

		<input type="submit" value="Submit">
	</form>

	{% if result %}
		<h2>Prediction Results:</h2>
		<p>Natural Conception Probability: {{ result.natural_prob }}</p>
		<p>IUI Conception Probability: {{ result.iui_prob }}</p>
		<p>IVF Conception Probability: {{ result.ivf_prob }}</p>
		<p>Natural Conception Time: {{ result.natural_time }} months</p>
		<p>IUI Conception Time: {{ result.iui_time }} months</p>
		<p>IVF Conception Time: {{ result.ivf_time }} months</p>
	{% endif %}
</body>
</html>
